In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('DS_Sample_1.csv')

In [3]:
df.head()

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,10001.0,40.900749,0.818182,95.40,0.00,95.4,0.000000,0.166667,0.000000,0.083333,0.000000,0.0,2.0,1000.0,201.802084,139.509787,0.000000,12.0
1,10002.0,3202.467416,0.909091,0.00,0.00,0.0,6442.945483,0.000000,0.000000,0.000000,0.250000,4.0,0.0,7000.0,4103.032597,1072.340217,0.222222,12.0
2,10003.0,2495.148862,1.000000,773.17,773.17,0.0,0.000000,1.000000,1.000000,0.000000,0.000000,0.0,12.0,7500.0,622.066742,627.284787,0.000000,12.0
3,10004.0,1666.670542,0.636364,1499.00,1499.00,0.0,205.788017,0.083333,0.083333,0.000000,0.083333,1.0,1.0,7500.0,0.000000,NaN,0.000000,12.0
4,10005.0,817.714335,1.000000,16.00,16.00,0.0,0.000000,0.083333,0.083333,0.000000,0.000000,0.0,1.0,1200.0,678.334763,244.791237,0.000000,12.0


# 전처리

In [6]:
df['CREDIT_LIMIT'] = df['CREDIT_LIMIT'].fillna(value=df['CREDIT_LIMIT'].mean())
df['MINIMUM_PAYMENTS'] = df['MINIMUM_PAYMENTS'].fillna(value=df['MINIMUM_PAYMENTS'].mean())

In [8]:
card1 = df.copy()

# 1

In [13]:
card1.groupby('TENURE')[['BALANCE', 'CREDIT_LIMIT']].corr().iloc[::2, -1].max().round(2)

0.95

# 2
# Min-Max Scaler , Standard Scaler
# model -> fit -> predict -> 평가

## 2-1

In [14]:
from sklearn.preprocessing import StandardScaler

In [15]:
df2 = card1.copy()

In [18]:
scaler = StandardScaler().fit(X=df2.iloc[:, 1:])

In [28]:
df_scaled = pd.DataFrame(scaler.transform(df2.iloc[:, 1:]),\
                         columns=df2.iloc[:, 1:].columns)

# pd.DataFrame(StandardScaler().fit_transform(X=df2.iloc[:, 1:]),\
#              columns=df2.iloc[:, 1:].columns)

## 2-2

In [29]:
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import KMeans

In [37]:
for i in [2, 3, 4, 5]:
    model = KMeans(n_clusters=i, random_state=1234).fit(X=df_scaled)
    pred = model.predict(X=df_scaled)
    score = silhouette_score(df_scaled, pred)
    print(i, score)

2 0.3075281530456079
3 0.19636128772937608
4 0.20715098494639902
5 0.19274056144483248


## 2-3

In [38]:
model = KMeans(n_clusters=2, random_state=1234).fit(X=df_scaled)
pred = model.predict(X=df_scaled)

In [39]:
df2['cluster'] = pred

In [43]:
df2.groupby('cluster')['ONEOFF_PURCHASES'].mean().max().round(2)

3946.19

# 3

## 3-1

In [44]:
train = card1[card1['CUST_ID'] % 4 != 0]
test = card1[card1['CUST_ID'] % 4 == 0]

## 3-2

In [45]:
from sklearn.tree import DecisionTreeRegressor

In [48]:
col = ['BALANCE', 'BALANCE_FREQUENCY', 'PURCHASES',
       'INSTALLMENTS_PURCHASES', 'CASH_ADVANCE',
       'PURCHASES_FREQUENCY', 'ONEOFF_PURCHASES_FREQUENCY',
       'PURCHASES_INSTALLMENTS_FREQUENCY', 'CASH_ADVANCE_FREQUENCY',
       'CASH_ADVANCE_TRX', 'PURCHASES_TRX', 'CREDIT_LIMIT', 'PAYMENTS',
       'MINIMUM_PAYMENTS', 'PRC_FULL_PAYMENT', 'TENURE']
len(col)

16

In [49]:
model = DecisionTreeRegressor(random_state=1234)\
            .fit(X=train[col], y=train['ONEOFF_PURCHASES'])

## 3-3

In [51]:
pred = model.predict(X=test[col])

In [52]:
from sklearn.metrics import mean_squared_error

In [54]:
round(mean_squared_error(y_true=test['ONEOFF_PURCHASES'], y_pred=pred) ** 0.5, 1)

2383.8